# Foresight 3D Costmap Demo

Visualize the 3D voxel costmap for visually impaired navigation. Synthetic obstacles: tables, doorways, ground objects.

In [ ]:
import sys
from pathlib import Path
root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(root))
import numpy as np
from foresight import VoxelCostmap, GROUND_LAYER, TORSO_LAYER, HEAD_LAYER

## 1. Create costmap and add synthetic obstacles

In [ ]:
cm = VoxelCostmap(origin=(0, 0, 0))

# Table (torso height)
cm.add_obstacle_box(2, 2, 0.7, 3, 4, 0.75)

# Low shelf (ankle level)
cm.add_obstacle_box(5, 1, 0, 6, 2, 0.15)

# Hanging sign (head height)
cm.add_obstacle_box(4, 5, 2.0, 5, 6, 2.2)

# Doorway frame - low ceiling (head clearance)
cm.add_obstacle_box(7, 3, 1.9, 8, 5, 2.5)

# Pillar in center
cm.add_obstacle_box(4.5, 4.5, 0, 5.5, 5.5, 2.8)

print("Costmap created with obstacles at ankle, torso, and head heights")

## 2. Interactive 3D visualization (Plotly)

In [ ]:
import plotly.graph_objects as go

occ = cm.get_occupied_voxels(include_inflated=True)

# Obstacles + inflation buffer
fig = go.Figure(data=[
    go.Scatter3d(
        x=occ[:, 0], y=occ[:, 1], z=occ[:, 2],
        mode='markers',
        marker=dict(size=4, color='rgba(200,50,50,0.7)', symbol='cube'),
        name='Obstacles + inflation'
    )
])

fig.update_layout(
    title='Foresight 3D Costmap - Obstacles at Head, Torso, Ankle',
    scene=dict(
        xaxis_title='X (m)', yaxis_title='Y (m)', zaxis_title='Z (m)',
        aspectmode='data',
        zaxis=dict(range=[0, 3])
    ),
    height=600,
    margin=dict(l=0, r=0, b=0, t=40)
)
fig.show()

## 3. 2D slice views (height layers)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

layers = [GROUND_LAYER, TORSO_LAYER, HEAD_LAYER]
titles = ['Ground (0-0.3m) ankle', 'Torso (0.3-1.8m)', 'Head (1.8-2.5m) overhead']

for ax, (layer, title) in zip(axes, zip(layers, titles)):
    sl = cm.get_layer_slice(layer)
    im = ax.imshow(sl.T, origin='lower', cmap='RdYlGn_r', vmin=0, vmax=100,
                   extent=[0, 20, 0, 20], aspect='equal')
    ax.set_title(title)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')

plt.colorbar(im, ax=axes, shrink=0.6, label='Cost (0=free, 100=occupied)')
plt.tight_layout()
plt.show()

## 4. Point cloud update (simulate SLAM input)

In [ ]:
# Simulate point cloud from sensor at origin looking forward
np.random.seed(42)
n = 500
points = np.column_stack([
    np.random.uniform(1, 8, n),
    np.random.uniform(1, 8, n),
    np.random.uniform(0.1, 2.5, n),
])

cm2 = VoxelCostmap()
cm2.add_point_cloud(points, sensor_origin=(0, 0, 1.5), mark_free=True)

occ2 = cm2.get_occupied_voxels(include_inflated=True)
fig2 = go.Figure(data=[
    go.Scatter3d(x=occ2[:, 0], y=occ2[:, 1], z=occ2[:, 2],
                 mode='markers', marker=dict(size=3, color='steelblue', opacity=0.6))
])
fig2.update_layout(title='Costmap from point cloud + ray tracing',
                   scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
fig2.show()